In [88]:
import pandas as pd
import numpy as np
import nltk
nltk.download('vader_lexicon')
import ast
from ast import literal_eval
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import FeatureUnion
import xgboost as xgb
import joblib

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import classification_report

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/zujiewu/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [31]:
df = pd.read_csv('/Users/zujiewu/Desktop/Zujie’s MacBook Air/ISE350/project/data/cleaned_data.csv')

In [32]:
df = df.iloc[:,1:]

In [33]:
df

,hashtags,tweet
0,covid19,"['covid19', 'era', 'normal', 'influenza', 'sea..."
1,greenpass,"['covid19', 'era', 'normal', 'influenza', 'sea..."
2,greenpass,"['brussel', 'march', 'greenpass', 'novaccinepa..."
3,novaccinepassports,"['brussel', 'march', 'greenpass', 'novaccinepa..."
4,novaccinemandates,"['brussel', 'march', 'greenpass', 'novaccinepa..."
...,...,...
30312,covidiots,"['dr', 'fleme', 'covid', 'caus', 'solut', 'cov..."
30313,vaccinesideeffects,"['dr', 'fleme', 'covid', 'caus', 'solut', 'cov..."
30314,vaccinepassport,"['dr', 'fleme', 'covid', 'caus', 'solut', 'cov..."
30315,vaccinemandate,"['dr', 'fleme', 'covid', 'caus', 'solut', 'cov..."


**compute compond score for each row by VADER model**

In [36]:
#make hashtags into lists
l = []
s = []
for index, row in df.iterrows():
    ht = row['tweet']
    l.append(ast.literal_eval(ht))
    s.append(row['hashtags'])

In [37]:
df_new = pd.DataFrame()
df_new['hashtags'] = s
df_new['tweet'] = l

In [45]:
type(df_new.tweet[0])

list

In [46]:
#convert list to string
df_new['text'] = df_new['tweet'].apply(lambda x: ' '.join(x))

In [47]:
df_new

,hashtags,tweet,text
0,covid19,"[covid19, era, normal, influenza, season, lot,...",covid19 era normal influenza season lot peopl ...
1,greenpass,"[covid19, era, normal, influenza, season, lot,...",covid19 era normal influenza season lot peopl ...
2,greenpass,"[brussel, march, greenpass, novaccinepassport,...",brussel march greenpass novaccinepassport nova...
3,novaccinepassports,"[brussel, march, greenpass, novaccinepassport,...",brussel march greenpass novaccinepassport nova...
4,novaccinemandates,"[brussel, march, greenpass, novaccinepassport,...",brussel march greenpass novaccinepassport nova...
...,...,...,...
30312,covidiots,"[dr, fleme, covid, caus, solut, covid19, covid...",dr fleme covid caus solut covid19 covid1984 co...
30313,vaccinesideeffects,"[dr, fleme, covid, caus, solut, covid19, covid...",dr fleme covid caus solut covid19 covid1984 co...
30314,vaccinepassport,"[dr, fleme, covid, caus, solut, covid19, covid...",dr fleme covid caus solut covid19 covid1984 co...
30315,vaccinemandate,"[dr, fleme, covid, caus, solut, covid19, covid...",dr fleme covid caus solut covid19 covid1984 co...


In [11]:
sid = SentimentIntensityAnalyzer()

In [48]:
#compute compond score for each row by VADER model
df_new['compound'] = df_new['text'].apply(lambda x:sid.polarity_scores(x)['compound'])

In [49]:
df_new['length'] = df_new['text'].apply(lambda x:len(x))

In [50]:
df_new

,hashtags,tweet,text,compound,length
0,covid19,"[covid19, era, normal, influenza, season, lot,...",covid19 era normal influenza season lot peopl ...,-0.6597,159
1,greenpass,"[covid19, era, normal, influenza, season, lot,...",covid19 era normal influenza season lot peopl ...,-0.6597,159
2,greenpass,"[brussel, march, greenpass, novaccinepassport,...",brussel march greenpass novaccinepassport nova...,0.0000,180
3,novaccinepassports,"[brussel, march, greenpass, novaccinepassport,...",brussel march greenpass novaccinepassport nova...,0.0000,180
4,novaccinemandates,"[brussel, march, greenpass, novaccinepassport,...",brussel march greenpass novaccinepassport nova...,0.0000,180
...,...,...,...,...,...
30312,covidiots,"[dr, fleme, covid, caus, solut, covid19, covid...",dr fleme covid caus solut covid19 covid1984 co...,0.0000,223
30313,vaccinesideeffects,"[dr, fleme, covid, caus, solut, covid19, covid...",dr fleme covid caus solut covid19 covid1984 co...,0.0000,223
30314,vaccinepassport,"[dr, fleme, covid, caus, solut, covid19, covid...",dr fleme covid caus solut covid19 covid1984 co...,0.0000,223
30315,vaccinemandate,"[dr, fleme, covid, caus, solut, covid19, covid...",dr fleme covid caus solut covid19 covid1984 co...,0.0000,223


**Preprocess data with tf-idf**

In [83]:
#split X Y (straitified)
def load_data(data):
    x = data['text'].tolist()
    y = data['hashtags'].tolist()
    return x, y

X, y = load_data(df_new)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify = y)

In [84]:
X_train

['look omicron govern use forc peopl vaccin report symptom mild yet govern keep creat panic vaccinemand vaccin usa american trump2024tosaveamerica',
 'peopl still bother selv listen cyril ramaphosa 😳😳😳 vaccinemand httpstcohppaygoh8i',
 'antivaxx replic fals claim shot 14 day die canadian data show complet fals break continu make whatev make feel justifi avoid vaccin cdnpoli vaccinemand bcpoli httpstco64y0k7sfkk',
 'that’ why… greenpass httpstcoae7cnjkgk6',
 'look vaccin mandat option work long run proven work altern power see consid vaccinemand',
 'mother allow child breast kid insid restaur novemb greenpass health right draghi apartheid',
 'happen due vaccinemand induc nycschool staff shortag staff fault leadership 52 chamber forc thousand u nyccouncil nysednew uft someth teacherchoic httpstcoynuicwgbft',
 'period februari 1 2021 – august 13 2021 protect infect declin ⛔janssen mar 92 aug 3 ⛔pfizer mar 95 aug 50 ⛔moderna mar 91 aug 64 vaccin vaccin covid19 covid19 vaccinemand covid19uk

**Feature union to cmobine text feature and numerical feature**

In [81]:
#load model
loaded_model = joblib.load('/Users/zujiewu/Desktop/Zujie’s MacBook Air/ISE350/project/Untitled Folder/xgb_tfidf.sav')

In [86]:
def get_score(x):
    return np.array([sid.polarity_scores(t)['compound'] for t in x]).reshape(-1, 1)
def get_text_length(x):
    return np.array([len(t) for t in x]).reshape(-1, 1)

classifier = Pipeline([
    ('features', FeatureUnion([
        ('text', Pipeline([
            ('vectorizer', TfidfVectorizer(max_features = 100)),
            
        ])),
        ('length', Pipeline([
            ('senti', FunctionTransformer(get_score, validate=False)),
            ('count', FunctionTransformer(get_text_length, validate=False)),
        ]))
    ])),
    ('clf', loaded_model)])

classifier.fit(X_train, y_train)
predicted = classifier.predict(X_test)
predicted

/Users/zujiewu/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[01:20:30] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


array(['vaccinemandate', 'covid', 'vaccinemandate', ..., 'vaccinemandate',
       'protests', 'coronavirus'], dtype='<U26')

In [89]:
print(classification_report(y_test, predicted, labels = ['vaccinemandate', 'vaccinepassport','greenpass','covid']))

                 precision    recall  f1-score   support

 vaccinemandate       0.50      0.85      0.63      1653
vaccinepassport       0.49      0.63      0.55       561
      greenpass       0.53      0.65      0.58       210
          covid       0.12      0.03      0.05       155

      micro avg       0.50      0.74      0.59      2579
      macro avg       0.41      0.54      0.45      2579
   weighted avg       0.48      0.74      0.57      2579



**tf-idf vec with other compound score and sentence length does not perform better than tf-idf vec only** 